In [1]:
CELLS = [(q, r) for q in range(-3, +3 + 1) for r in range(-3, +3 + 1) if -q - r in range(-3, +3 + 1)]

In [18]:
import tensorflow as tf
import numpy as np
from keras import models, layers
from keras.models import load_model
import json
import pickle
import os
from os.path import join
import subprocess
import json
import VanGame.utils as utils

In [404]:
sequence = sorted([p for p in CELLS])
sequence

[(-3, 0),
 (-3, 1),
 (-3, 2),
 (-3, 3),
 (-2, -1),
 (-2, 0),
 (-2, 1),
 (-2, 2),
 (-2, 3),
 (-1, -2),
 (-1, -1),
 (-1, 0),
 (-1, 1),
 (-1, 2),
 (-1, 3),
 (0, -3),
 (0, -2),
 (0, -1),
 (0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, -3),
 (1, -2),
 (1, -1),
 (1, 0),
 (1, 1),
 (1, 2),
 (2, -3),
 (2, -2),
 (2, -1),
 (2, 0),
 (2, 1),
 (3, -3),
 (3, -2),
 (3, -1),
 (3, 0)]

In [6]:
a = np.array([[1]*39])
b = [500]

In [464]:
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(1,activation='relu', input_shape=(8,)))
    #model.add(layers.Dense(200, activation='relu'))
    #model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse')
    return model

In [465]:
model = build_model()

In [324]:
try:
    model = load_model('VanGame/my_model.h5')
except:
    print("none model found")
    model = build_model()

In [466]:
!ls 'rec'

'[2zzz]1556610247.0caoredjiba.txt'


In [24]:
def bash_call(c:[str]) -> None:
    res = subprocess.check_output(c)
    ls = []
    for line in res.splitlines():
        line = line.decode('utf-8')
        ls.append(line)
    return ls

In [25]:
def join_list(l):
    res = ""
    for i in l:
        res += i + "/"
    
    res = res[:-1]
    return res

In [26]:
def load_logs():
    datapath = join(os.getcwd(),'rec/')
    files = bash_call(['ls', datapath])
    return files

In [457]:
def PG_log(filename, condition=None, value_check=False):
    
    if (condition != None) and (condition not in filename):
        return False, False
    
    #print(filename)
    states, color = utils.load_l('rec/'+filename)
    
    train = []
    value = []
    
    for s in states:
        # print(s)
        board = []
        # add board
        for cord in sequence:
            # does not include the board
            break
            board.append(s['board'][cord])
        # color 
        board.append(color)
        # utility = [heuristic diff, my pieces difference, danger pieces]
        board.append(s['turns'])
        board += s['utility']
        train.append(board)
        value.append(s['rew'])
        
    if value_check:
        print(value)
        
    value.reverse()
    decay = 0.95
    curr_decay = 0.95
    for i in range(1, len(value)):
        value[i] += value[i-1] * curr_decay
        curr_decay *= decay
    
    value.reverse()
    
    if value_check:
        print(value)
    
    post_value = np.array(value)
    
    # normalize the value
    # post_value = (post_value - post_value.mean()) / post_value.std()
    
    if value_check:
        # print(post_value)
        pass
    
    return train, value

In [463]:
t,v = PG_log('[2zzz]1556610247.0caoredjiba.txt', value_check=True)

[0, -6.0, 0, 0, 0, -6.0, 0, 0, -30, -6.0, -30, -25, -6.0, -25, -30, 0, -6.0, 0, -6.0, -6.0, -30, 0, 0, 0, 0, 0, -430]
[-1.597782886715534, -6.063229788813119, -0.2279458023656384, -0.7806656530389667, -2.539932355561967, -7.850599713758793, -5.433964284024144, -15.158096718318072, -40.16947663007882, -25.601989045992852, -46.88121431768577, -38.35535431060742, -28.827125231794692, -46.80807137818523, -42.48253180191963, -23.100395619361244, -40.612503290758134, -57.80919970427557, -91.72432185146451, -129.21567954260536, -176.4414993038894, -199.21522896869385, -257.45688387250283, -316.08951296874994, -368.67125, -408.5, -430]
8


In [441]:
!ls rec

'[2zzz]1556610247.0caoredjiba.txt'


In [331]:
def reload_model():
    del model  # deletes the existing model

    # returns a compiled model
    # identical to the previous one
    model = load_model('my_model.h5')

In [384]:
def save_model():
    model.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'
    with open('trained_model', 'wb') as f:
        pickle.dump(model.get_weights(), f)

In [351]:
def train(save=True, epochs):
    logfs = load_logs()
    train_set = []
    target_set = []
    
    for logf in logfs:
        train, value = PG_log(logf)
        if train:
            #print(len(train), len(value))
            #print(train[-1])
            train_set += train
            # the target shouldnt be calculate like this, should use bootstrapping
            target_set += value

    model.fit(np.array(train_set), target_set, epochs=epochs, batch_size=1)
    if save:
        print("saved")
        save_model()   

In [352]:
train()

Epoch 1/2
323/323 [==============================] - 2s 6ms/step - loss: 0.7966
Epoch 2/2
323/323 [==============================] - 1s 3ms/step - loss: 0.5005
saved


In [467]:
save_model()

In [321]:
del model

In [468]:
model.predict(np.array([[1]*8,[0.5]*8]))

array([[1.6910833 ],
       [0.84554166]], dtype=float32)

In [52]:
def clean_log(logs):
    !rm '/Users/xinyaoniu/Documents/COMP30024-AI/Project2/rec/' *

In [53]:
clean_log(logfs)

rm: cannot remove '/Users/xinyaoniu/Documents/COMP30024-AI/Project2/rec/': Is a directory
rm: cannot remove '__pycache__': Is a directory


In [110]:
os.getcwd()

'/Users/xinyaoniu/Documents/COMP30024-AI/Project2'

In [146]:
!rm -v ./rec/*

removed './rec/[1win]1556549315.0caoredjiba.txt'
removed './rec/[1win]1556549321.0caoredjiba.txt'
removed './rec/[2zzz]1556549305.0caoredjiba.txt'
removed './rec/[2zzz]1556549307.0caoredjiba.txt'
removed './rec/[2zzz]1556549308.0caoredjiba.txt'
removed './rec/[2zzz]1556549309.0caoredjiba.txt'
removed './rec/[2zzz]1556549311.0caoredjiba.txt'


In [353]:
def full_train_cycle(n, epochs=2):
    for i in range(n):
        !python3 -m referee VanGame HardCode HardCode -l logtry -v 0
    train(epochs=epochs)
    !rm -v ./rec/*

In [488]:
for i in range(30):
    print("********** starting round", i, "**********")
    full_train_cycle(2, epochs=2)

********** starting round 0 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
57/57 [==============================] - 0s 2ms/step - loss: 16798.9150
Epoch 2/2
57/57 [==============================] - 0s 2ms/step - loss: 16749.6797
saved
removed './rec/[2zzz]1556619785.0caoredjiba.txt'
removed './rec/[2zzz]1556619787.0caoredjiba.txt'
********** starting round 1 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Red
Epoch 1/2
153/153 [==============================] - 0s 1ms/step - loss: 1188.7293
Epoch 2/2
153/153 [==============================] - 0s 2ms/step - loss: 1170.4208
saved
removed './rec/[1win]1556619790.0caoredjiba.txt'
removed './rec/[1win]1556619793.0caoredjiba.txt'
********** starting round 2 **********
* NOT

* winner: Blue
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
45/45 [==============================] - 0s 1ms/step - loss: 34221.0371
Epoch 2/2
45/45 [==============================] - 0s 1ms/step - loss: 34214.4364
saved
removed './rec/[2zzz]1556619852.0caoredjiba.txt'
removed './rec/[2zzz]1556619853.0caoredjiba.txt'
********** starting round 18 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
Epoch 1/2
48/48 [==============================] - 0s 2ms/step - loss: 29641.4379
Epoch 2/2
48/48 [==============================] - 0s 2ms/step - loss: 29598.2559
saved
removed './rec/[2zzz]1556619854.0caoredjiba.txt'
removed './rec/[2zzz]1556619855.0caoredjiba.txt'
********** starting round 19 **********
* NOTE: unable to measure memory usage on this platform (try dimefox)
* winner: Green
* NOTE: unable to measure

In [473]:
# utility = [color, turns, heuristic diff, my pieces difference, danger pieces, red exit, green exit, blue exit]
model.get_weights()

[array([[-0.69869846],
        [ 0.13379724],
        [ 0.26100662],
        [ 0.6259155 ],
        [-0.8565208 ],
        [ 1.0463128 ],
        [-0.5763864 ],
        [ 0.24955097]], dtype=float32),
 array([-0.6723143], dtype=float32),
 array([[0.4011983]], dtype=float32),
 array([-1.0327048], dtype=float32)]

In [475]:
model.get_weights()

[array([[-0.70649874],
        [ 0.15657607],
        [ 0.09836416],
        [ 0.73236334],
        [-0.88198453],
        [ 1.1993474 ],
        [-0.54759383],
        [ 0.23956929]], dtype=float32),
 array([-0.68011457], dtype=float32),
 array([[0.4796768]], dtype=float32),
 array([-1.5186025], dtype=float32)]

In [477]:
model.get_weights()

[array([[-0.81592125],
        [ 0.10808953],
        [-0.08458312],
        [ 0.85356146],
        [-1.0024132 ],
        [ 1.3659197 ],
        [-0.61527216],
        [ 0.16224095]], dtype=float32),
 array([-0.7895371], dtype=float32),
 array([[0.5366073]], dtype=float32),
 array([-1.8857365], dtype=float32)]

In [479]:
model.get_weights()

[array([[-0.828572  ],
        [ 0.15462501],
        [-0.33071476],
        [ 1.0068716 ],
        [-1.102255  ],
        [ 1.6234683 ],
        [-0.5860915 ],
        [ 0.17780784]], dtype=float32),
 array([-0.8021878], dtype=float32),
 array([[0.68850666]], dtype=float32),
 array([-2.2046783], dtype=float32)]

In [482]:
# utility = [color, turns, heuristic diff, my pieces difference, danger pieces, red exit, green exit, blue exit]
model.get_weights()

[array([[-0.9445977 ],
        [ 0.20689423],
        [-0.9093882 ],
        [ 1.5389959 ],
        [-1.4014518 ],
        [ 2.177783  ],
        [-0.53679943],
        [ 0.12599692]], dtype=float32),
 array([-0.9182137], dtype=float32),
 array([[1.1451691]], dtype=float32),
 array([-4.3304048], dtype=float32)]

In [484]:
# utility = [color, turns, heuristic diff, my pieces difference, danger pieces, red exit, green exit, blue exit]
model.get_weights()

[array([[-1.1199762 ],
        [ 0.13880344],
        [-1.238538  ],
        [ 1.991631  ],
        [-1.6170409 ],
        [ 2.5888567 ],
        [-0.5939308 ],
        [-0.00912558]], dtype=float32),
 array([-1.0935918], dtype=float32),
 array([[1.3876469]], dtype=float32),
 array([-6.6800756], dtype=float32)]

In [486]:
# utility = [color, turns, heuristic diff, my pieces difference, danger pieces, red exit, green exit, blue exit]
model.get_weights()

[array([[-1.2102667 ],
        [ 0.16690053],
        [-1.5944142 ],
        [ 2.5314486 ],
        [-1.8229241 ],
        [ 3.1320689 ],
        [-0.60356563],
        [-0.04025923]], dtype=float32),
 array([-1.1838824], dtype=float32),
 array([[1.7319276]], dtype=float32),
 array([-8.563517], dtype=float32)]

In [489]:
model.get_weights()

[array([[-1.2868927 ],
        [ 0.14712715],
        [-1.8580728 ],
        [ 2.988167  ],
        [-2.1074595 ],
        [ 3.4834957 ],
        [-0.5701345 ],
        [-0.06940529]], dtype=float32),
 array([-1.2605083], dtype=float32),
 array([[2.0096612]], dtype=float32),
 array([-10.228373], dtype=float32)]

In [490]:
qwer = np.array([0.1,0.2,0.3,0.4])

In [491]:
np.argmax(qwer)

3

# CNN attamp

In [226]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
md = Sequential()
#add model layers
md.add(Conv2D(1, kernel_size=3, activation='relu', input_shape=(28,28,1), use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
#md.add(layers.MaxPooling2D(2,2))
#md.add(Conv2D(64, kernel_size=3, activation='relu', use_bias=False))
md.add(Flatten())
md.compile(optimizer='adam', loss='mse')

In [227]:
for i in md.get_weights():
    print(i.shape)

(3, 3, 1, 1)


In [228]:
md.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 26, 26, 1)         9         
_________________________________________________________________
flatten_9 (Flatten)          (None, 676)               0         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________


In [229]:
md.predict(np.array([[[[1]*28]*28]]).reshape(1,28,28,1))

array([[-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
        -0., -0., -0., -0., -0., -0., -0., -0., -0.

In [244]:
asdf = md.get_weights()[0].reshape(3,3)

In [230]:
wds = md.get_weights()

In [231]:
convp = wds[0]

In [232]:
convp.shape

(3, 3, 1, 1)

In [233]:
c = convp.reshape(1, 3, 3, 1)

In [234]:
d = c[0].reshape(3,3)

In [235]:
import scipy.signal as signal

In [236]:
a = np.array([[[1]*28]*28]).reshape(28,28)

In [245]:
e = signal.convolve2d(a, asdf, mode='valid')

In [246]:
e

array([[-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036],
       [-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036],
       [-0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
        -0.5030036, -0.5030036, -0.5030036, -0.5030036, -0.5030036,
      

In [206]:
signal.convolve(e, )

ValueError: volume and kernel should have the same dimensionality